In [ ]:
import pandas as pd
from tqdm import tqdm
df = pd.read_parquet('postech_company_project.parquet')


In [ ]:
df.head()

In [ ]:
df = df.drop(['YEAR_YYYY', 'MONTH_YYYYMM'], axis=1)

columns_to_drop = ['PARTS_NO1', 'PARTS_NO2', 'PARTS_NO3', 'PARTS_NO4', 'PARTS_NO5',
                   'PREV_PARTS_NO1', 'PREV_PARTS_NO2', 'PREV_PARTS_NO3', 'PREV_PARTS_NO4', 'PREV_PARTS_NO5',
                   'PARTS_DESC1', 'PARTS_DESC2', 'PARTS_DESC3', 'PARTS_DESC4', 'PARTS_DESC5',
                   'PREV_PARTS_DESC1', 'PREV_PARTS_DESC2', 'PREV_PARTS_DESC3', 'PREV_PARTS_DESC4', 'PREV_PARTS_DESC5']

df = df.drop(columns_to_drop, axis=1)
columns_to_drop = ['KEY_PARTS1', 'KEY_PARTS2', 'KEY_PARTS3', 'KEY_PARTS4', 'KEY_PARTS5',
                   'PREV_KEY_PARTS1', 'PREV_KEY_PARTS2', 'PREV_KEY_PARTS3', 'PREV_KEY_PARTS4', 'PREV_KEY_PARTS5',
                   'KEY_PARTS_REMARK1', 'KEY_PARTS_REMARK2', 'KEY_PARTS_REMARK3', 'KEY_PARTS_REMARK4', 'KEY_PARTS_REMARK5',
                   'PREV_KEY_PARTS_REMARK1', 'PREV_KEY_PARTS_REMARK2', 'PREV_KEY_PARTS_REMARK3', 'PREV_KEY_PARTS_REMARK4', 'PREV_KEY_PARTS_REMARK5']

df = df.drop(columns_to_drop, axis=1)

print(df)

In [ ]:
df.head()

 "RECEIPT_NUMBER" 및 "PREV_RECEIPT_NUMBER" 열에서 "LGE_REC_" 부분 제거 및 문자열을 숫자로 변환
 기존 데이터를 바탕으로 다음 재수리가 발생하는지 확인 할때 숫자값만을 이용해서 인덱스로 빠르게 데이터 가공하기 위함

In [ ]:
df['RECEIPT_NUMBER'] = df['RECEIPT_NUMBER'].str.replace('LGE_REC_', '').astype(int, errors='ignore')
df['PREV_RECEIPT_NUMBER'] = df['PREV_RECEIPT_NUMBER'].str.replace('LGE_REC_', '').astype(int, errors='ignore')

In [ ]:
df.set_index('RECEIPT_NUMBER', inplace=True)

In [ ]:
df['NEXT_RECLAIM'] = 0  # "NEXT_RECLAIM" 열을 초기화

for index, row in tqdm(df.iterrows(), total=len(df)):
    prev_receipt_number = row['PREV_RECEIPT_NUMBER']
    if not pd.isna(prev_receipt_number):
        prev_receipt_number = int(prev_receipt_number)
        if prev_receipt_number in df.index:
            df.at[prev_receipt_number, 'NEXT_RECLAIM'] = 1

In [ ]:
print(df.columns)

각 데이터의 유니크값 확인

In [ ]:
unique_values = df.nunique()
for column_name, unique_count in unique_values.items():
    unique_data = df[column_name].unique()
    print(f"Column: {column_name}, Unique Count: {unique_count}")
    print(unique_data)
    print()

엔지니어 숫자 확인

In [ ]:
engineer_counts = df.groupby('CENTER_CODE')['ENGINEER_CODE'].unique()
print(engineer_counts)


1명이상의 엔지니어 숫자 확인

In [ ]:
result = df.groupby('CENTER_CODE').agg(
    ENGINEER_COUNT=('ENGINEER_CODE', 'nunique'),
    SUBSIDIARY_NAME=('SUBSIDIARY_NAME', 'first')
)

# 1개인 CENTER_CODE 제외
filtered_result = result[result['ENGINEER_COUNT'] > 1]
print(filtered_result)

센터별 엔지니어 숫자와 재수리 비율 확인

In [ ]:
result = df.groupby('CENTER_CODE').agg(
    ENGINEER_COUNT=('ENGINEER_CODE', 'nunique'),
    SUBSIDIARY_NAME=('SUBSIDIARY_NAME', 'first'),
    RECLAIM_SUM=('RECLAIM', 'sum')
)
result['RECLAIM_RATIO'] = result['RECLAIM_SUM'] / result['ENGINEER_COUNT']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.boxplot(x='RECLAIM_RATIO', data=result)
plt.xlabel('RECLAIM_RATIO')
plt.show()

In [ ]:
# 날짜 형식으로 변환
df['REPAIR_START_DATE'] = pd.to_datetime(df['REPAIR_START_DATE'], format='%Y%m%d')
df['REPAIR_END_DATE'] = pd.to_datetime(df['REPAIR_END_DATE'], format='%Y%m%d')

# 수리 걸린 시간 계산
df['REPAIR_DURATION'] = df['REPAIR_END_DATE'] - df['REPAIR_START_DATE']

엔지니어별 Load 계산

In [ ]:
def format_date(date_str):
    date_obj = pd.to_datetime(date_str, format='%Y%m%d')
    week_number = date_obj.strftime('%U')
    formatted_date = date_obj.strftime('%Y') + 'W' + week_number
    return formatted_date

df['REPAIR_START_DATE'] = df['REPAIR_START_DATE'].progress_apply(format_date)

In [ ]:
for index, row in tqdm(df.iterrows(), total=len(df)):
    week = row['REPAIR_START_DATE']
    engineer_code = row['ENGINEER_CODE']
    if engineer_code not in df.index:
        df.loc[engineer_code] = 0
    df.at[engineer_code, week] += 1